In [1]:
import base64
import io
import json
import os
import time

from IPython.core.display import HTML
from IPython.display import Audio
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from models.config import PreprocessingConfig
from models.tts.delightful_tts.delightful_tts_refined import DelightfulTTS
from models.vocoder.univnet import UnivNet
from training.loss import Metrics
from training.preprocess import TacotronSTFT

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sample_rate = 22050

## Add the checkpoints to the `/checkpoints` folder and choose the appropriate version

In [2]:
# Checkpoints to use for demo
checkpoint_name = "logs_new_training_libri-360-swa_19_sec_epoch=190-step=70670"
checkpoint_univnet = "epoch=23-step=11616"


# checkpoint_path = f"./logs_100/{checkpoint_name}.ckpt"
# checkpoint_path = f"./logs_360_energy/{checkpoint_name}.ckpt"
# checkpoint_path = f"./logs/{checkpoint_name}.ckpt"
checkpoint_path = f"./checkpoints/{checkpoint_name}.ckpt"
checkpoint_path_univnet = f"./checkpoints/{checkpoint_univnet}.ckpt"

# Dataset url to use for demo
# dataset_url = "train-clean-100"
dataset_url = "train-clean-100"

# text_tts = """
# Casablanca: “But what about us?”
# “We’ll always have Paris.”
# The Wizard of Oz: “Lions? And Tigers? And Bears?”
# “Oh my!”
# Star Wars (A New Hope): “He’s almost in range.”
# “That’s no moon; it’s a space station.”
# Love Story: “Jenny, I’m sorry.”
# “Don’t. Love means never having to say you’re sorry.”
# No Country for Old Men: “Look, I need to know what I stand to win.”
# “Everything.”
# Forrest Gump: “I thought I’d try out my sea legs.”
# “But you ain’t got no legs, Lieutenant Dan.”
# Toy Story: “Buzz, you’re flying!”
# “This isn’t flying; this is falling with style.”
# As the snake shook its head, a deafening shout behind Harry made both of them jump.
# ‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
# """

text_tts = """
Casablanca: “But what about us?”
“We’ll always have Paris.”
The Wizard of Oz: “Lions? And Tigers? And Bears?”
“Oh my!”
"""

### Load from the checkpoint

In [3]:
model = DelightfulTTS.load_from_checkpoint(checkpoint_path, strict=False).to(device)
model.eval()

model.vocoder_module = UnivNet()
model.vocoder_module.eval()

print(f"Loaded checkpoint: {checkpoint_path}")

preprocess_config = PreprocessingConfig("english_only")
tacotronSTFT = TacotronSTFT(
    filter_length=preprocess_config.stft.filter_length,
    hop_length=preprocess_config.stft.hop_length,
    win_length=preprocess_config.stft.win_length,
    n_mel_channels=preprocess_config.stft.n_mel_channels,
    sampling_rate=preprocess_config.sampling_rate,
    mel_fmin=preprocess_config.stft.mel_fmin,
    mel_fmax=preprocess_config.stft.mel_fmax,
    center=False,
)
tacotronSTFT = tacotronSTFT.to(device)

/home/you/anaconda3/envs/tts_framework/lib/python3.11/site-packages/lightning/pytorch/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.2.0.post0, which is newer than your current Lightning version: v2.1.2


Loaded checkpoint: ./checkpoints/logs_new_training_libri-360-swa_19_sec_epoch=190-step=70670.ckpt


### Load the speakers list

In [4]:
def plot_spectrogram(mel: np.ndarray):
    r"""Plots the mel spectrogram."""
    plt.figure(dpi=80, figsize=(10, 3))

    img = librosa.display.specshow(mel, x_axis="time", y_axis="mel", sr=sample_rate)
    plt.title("Spectrogram")
    plt.colorbar(img, format="%+2.0f dB")

    # Save the plot to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)

    # Convert the BytesIO object to a base64 string
    img_str = base64.b64encode(buf.read()).decode("utf-8")
    plt.close()

    return img_str

def gen_table(text_demo_with_metrics: list):
    # Initialize an empty string to store the HTML
    html = "<table border='1'>"

    html += f"<h4>TTS: </h4> {text_tts}"

    html += "<h4>Speakers: </h4>"
    html += """<tr>
        <th>SpeakerID</th>
        <th>Speaker Name</th>
        <th>Gender</th>
        <th>Subset</th>
        <th>Audio</th>
        <th>InfTime</th>
        <th>ermr</th>
        <th>jitter</th>
        <th>shimmer</th>
        <th>spec</th>
    </tr>"""

    for row in text_demo_with_metrics:
        # Speaker info
        speaker_id = row["speaker_id"]
        speaker_name = row["speaker_name"]
        gender = row["gender"]
        subset = row["subset"]
        execution_time = row["execution_time"]

        # Waveforms and mel spectrogram
        wav = row["wav"]
        mel = row["mel"]

        # Metrics
        ermr = row["ermr"]
        jitter = row["jitter"]
        shimmer = row["shimmer"]

        # Round the metrics to 3 decimal places
        metrics = [round(x, 3) for x in [ermr, jitter, shimmer]]

        audio = Audio(wav, rate=sample_rate, autoplay=False)

        # Generate the spectrogram plot
        fig = plot_spectrogram(mel)

        # Add a row to the HTML table
        html += f"""<tr>
            <td>{speaker_id}</td>
            <td>{speaker_name}</td>
            <td>{gender}</td>
            <td>{subset}</td>
            <td>{audio._repr_html_()}</td>
            <td>{execution_time}</td>
            <td>{metrics[0]}</td>
            <td>{metrics[1]}</td>
            <td>{metrics[2]}</td>
            <td><img src='data:image/png;base64,{fig}' /></td>
        </tr>"""

    # Close the HTML table
    html += "</table>"

    return HTML(html)


In [5]:
def gen_batch_table(batch: list, batch_idx: int):
    df = pd.DataFrame(batch)

    # Get the unique cluster labels
    df_ermr_sorted = df.sort_values(by="ermr")
    metrics = ["ermr", "jitter", "shimmer"]

    demo_dirname = f"logs/{dataset_url}_{checkpoint_name}"
    os.makedirs(demo_dirname, exist_ok=True)

    # Add header
    result = f"""
    <h3>Chunk #{batch_idx}: </h3>
    {df_ermr_sorted[["speaker_id", *metrics]].describe().to_html()}
    <h3>Chunk #{batch_idx} audio data: </h3>
    """

    # Generate and display the table for the chunk
    # for _, row in df_ermr_sorted.iterrows():
    result += gen_table(df_ermr_sorted.to_dict("records")).data # type: ignore

    # Save result as HTML
    with open(f"{demo_dirname}/chunk_{batch_idx}.html", "w") as f:
        f.write(result)


In [ ]:
# text_tts = """
# Casablanca: “But what about us?”
# “We’ll always have Paris.”
# The Wizard of Oz: “Lions? And Tigers? And Bears?”
# “Oh my!”
# Star Wars (A New Hope): “He’s almost in range.”
# “That’s no moon; it’s a space station.”
# Love Story: “Jenny, I’m sorry.”
# “Don’t. Love means never having to say you’re sorry.”
# No Country for Old Men: “Look, I need to know what I stand to win.”
# “Everything.”
# Forrest Gump: “I thought I’d try out my sea legs.”
# “But you ain’t got no legs, Lieutenant Dan.”
# Toy Story: “Buzz, you’re flying!”
# “This isn’t flying; this is falling with style.”
# As the snake shook its head, a deafening shout behind Harry made both of them jump.
# ‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
# """

text_tts = """
The Wizard of Oz: “Lions? And Tigers? And Bears?”
Toy Story: “Buzz, you’re flying!”
As the snake shook its head, a deafening shout behind Harry made both of them jump.
‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
"""

with open("training/datasets/speaker_id_mapping_libri.json") as f:
    id_mapping = json.load(f)

batch_size = 200
model.vocoder_module.to(device)

metrics = Metrics()
text_demo_with_metrics = []

speakers_df = pd.read_csv(
    "./datasets_cache/LIBRITTS/LibriTTS/speakers.tsv",
    sep="\t",
    names=["READER", "GENDER", "SUBSET", "NAME"],
)

subsets = ["train-clean-100", "train-clean-360", "train-other-500"]

speakers = speakers_df[speakers_df['SUBSET'].isin(subsets)]

start_chunk = 12  # The chunk to start from
start_idx = start_chunk * batch_size  # The index to start from

for idx, speaker in enumerate(speakers.to_dict("records")):
    # Skip the speakers that have already been processed
    if idx < start_idx:
        continue

    speaker_id = speaker["READER"]
    speaker_id = id_mapping.get(str(speaker_id))
    speaker_name = speaker["NAME"]
    gender = speaker["GENDER"]
    subset = speaker["SUBSET"]

    with torch.no_grad():
        speaker_id_ = torch.tensor([int(speaker_id)], device=device)
        start_time = time.time()
        wav = model.forward(text_tts, speaker_id_)
        end_time = time.time()
        (
            ermr,
            jitter,
            shimmer,
        ) = metrics.wav_metrics(wav.unsqueeze(0))
        execution_time = round(end_time - start_time, 2)

        mel = tacotronSTFT.get_mel_from_wav(wav)

        text_demo_with_metrics.append({
            "speaker_id": speaker_id,
            "speaker_name": speaker_name,
            "gender": gender,
            "subset": subset,
            "wav": wav.detach().cpu().numpy(),
            "execution_time": execution_time,
            "mel": mel.detach().cpu().numpy(),
            "ermr": ermr,
            "jitter": jitter,
            "shimmer": shimmer,
        })

    if idx > 0 and idx % batch_size == 0:
        gen_batch_table(text_demo_with_metrics, idx // batch_size)
        text_demo_with_metrics = []

    torch.cuda.empty_cache()

gen_batch_table(text_demo_with_metrics, 999)
